In [122]:
import numpy as np

In [123]:
START_STATE = (0,0)
FOOD_STATE = (4,4)
ACTIONS = [(0,-1), (0,1), (-1,0), (1,0)]
p={}
p['specified'] = 0.7
p['right'] = 0.12
p['left'] = 0.12
p['sleepy'] = 0.06
FORBIDDEN_FURNITURES = [(2,1), (2,2), (2,3), (3,2)]
MONSTERS = [(0,3), (4,1)]
DELTA = 0.1
GAMMA = 0.925

In [124]:
def getTransitionProbabilities(action, p):
    if action == (0, -1):
        specified = (0, -1)
        left = (1, 0)
        right = (-1, 0)
    elif action == (0, 1):
        specified = (0, 1)
        left = (-1, 0)
        right = (1, 0)
    elif action == (-1, 0):
        specified = (-1, 0)
        left = (0, -1)
        right = (0, 1)
    else:
        specified = (1, 0)
        left = (0, 1)
        right = (0, -1)

    sleepy = (0, 0)
    return [(specified, p['specified']), (left, p['left']), (right, p['right']), (sleepy, p['sleepy'])]

In [125]:
def getReward(state):
    if state == FOOD_STATE:
        return 10
    elif state in MONSTERS:
        return -8
    else:
        return -0.05

In [126]:
def isValidState(i, j):
    if i < 0 or j < 0 or i >= 5 or j >= 5:
        return False
    if (i, j) in FORBIDDEN_FURNITURES:
        return False
    return True

In [127]:
def random_initial_state():
    while True:
        state = (np.random.randint(5), np.random.randint(5))
        if state not in FORBIDDEN_FURNITURES:
            return state

In [128]:
def generate_episode(policy_mat):
    episode = []
    state = random_initial_state()
    while state != FOOD_STATE:
        action = policy_mat[state[0]][state[1]]
        transitions = getTransitionProbabilities(action, p)
        
        next_state = (-1, -1)
        moves, probs = zip(*transitions)
        chosen_move = np.random.choice(len(moves), p=probs)
        move = moves[chosen_move]
        next_state = (state[0] + move[0], state[1] + move[1])
        if not isValidState(*next_state):
            next_state = state

        
        reward = getReward(next_state)
        episode.append((state, reward))
        state = next_state
    return episode

In [ ]:
def first_visit_monte_carlo(policy_mat, value_iteration_v):
    V = np.zeros((5, 5))
    visit_counts = {state: 0 for state in [(i, j) for i in range(5) for j in range(5) if (i, j) not in FORBIDDEN_FURNITURES]}
    max_norm = float('inf')
    trajectory_count = 0

    while max_norm > DELTA:
        episode = generate_episode(policy_mat)
        trajectory_count += 1
        visited_states = set()

        for t, (state, _) in enumerate(episode):
            if state not in visited_states:
                visited_states.add(state)
                G = 0
                for i, (_, reward) in enumerate(episode[t:]):
                    G += (GAMMA ** i) * reward

                visit_counts[state] += 1
                n = visit_counts[state]
                V[state[0], state[1]] = V[state[0], state[1]] + (G - V[state[0], state[1]]) / n

        max_norm = np.max(np.abs(V - value_iteration_v))

    return V, trajectory_count

In [ ]:
def every_visit_monte_carlo(policy_mat, value_iteration_v):
    V = np.zeros((5, 5))
    visit_counts = {state: 0 for state in [(i, j) for i in range(5) for j in range(5) if (i, j) not in FORBIDDEN_FURNITURES]}
    max_norm = float('inf')
    trajectory_count = 0

    while max_norm > DELTA:
        episode = generate_episode(policy_mat)
        trajectory_count += 1

        G = 0
        for t in reversed(range(len(episode))):
            state, reward = episode[t]
            G = reward + GAMMA * G

            visit_counts[state] += 1
            n = visit_counts[state]
            V[state[0], state[1]] = V[state[0], state[1]] + (G - V[state[0], state[1]]) / n

        max_norm = np.max(np.abs(V - value_iteration_v))

    return V, trajectory_count


In [131]:
policy_mat = [
    [(0, 1), (1, 0), (0, -1), (1, 0), (1, 0)],
    [(0, 1), (0, 1), (0, 1), (0, 1), (1, 0)],   
    [(-1, 0), None, None, None, (1, 0)],
    [(-1, 0), (0, -1), None, (0, 1), (1, 0)],
    [(-1, 0), (0, 1), (0, 1), (0, 1), None]
]

In [132]:
value_iteration_v = np.array([
    [2.6638, 2.9969, 2.8117, 3.6671, 4.8497],
    [2.9713, 3.5101, 4.0819, 4.8497, 7.1648],
    [2.5936, 0.0,    0.0,    0.0,    8.4687],
    [2.0992, 1.0849, 0.0,    8.6097, 9.5269],
    [1.0849, 4.9465, 8.4687, 9.5269, 0.0]
])

In [136]:
V_mc, num_trajectories = first_visit_monte_carlo(policy_mat, value_iteration_v)
print("Estimated Value Function using First-Visit Monte Carlo:\n", V_mc)
print("\nTotal trajectories needed for convergence (max-norm < 0.1):", num_trajectories)

Estimated Value Function using First-Visit Monte Carlo:
 [[2.64696838 2.98428491 2.75836578 3.56853442 4.85895907]
 [2.95802445 3.48637308 4.07076583 4.80205315 7.13297904]
 [2.58369027 0.         0.         0.         8.46087393]
 [2.05684464 1.17865945 0.         8.6272001  9.5266583 ]
 [1.1129969  4.95736015 8.48280033 9.54740927 0.        ]]

Total trajectories needed for convergence (max-norm < 0.1): 15313


In [137]:
V_mc, num_trajectories = every_visit_monte_carlo(policy_mat, value_iteration_v)
print("Estimated Value Function using Every-Visit Monte Carlo:\n", V_mc)
print("\nTotal trajectories needed for convergence (max-norm < 0.1):", num_trajectories)

Estimated Value Function using Every-Visit Monte Carlo:
 [[2.56451236 2.97033026 2.90205953 3.75829544 4.81889039]
 [3.01940378 3.51283498 4.09143634 4.87048023 7.15409821]
 [2.63731905 0.         0.         0.         8.46279127]
 [2.11932044 1.1074043  0.         8.60347848 9.52270901]
 [1.03631565 4.88847717 8.50112632 9.53591615 0.        ]]

Total trajectories needed for convergence (max-norm < 0.1): 14852
